[Table of Contents](../../index.ipynb)

# FRC Analytics with Python - Session 20
# Structured Query Language (SQL) - Part IV
**Last Updated: 25 November 2021**

Congratulations. You made it to the final SQL session. In this we'll learn how about multi-part and window queries.

## I. Notebook Setup

### A. If Using Google Colab
It's best if you clone the *pyclass_frc* Github repo and run this notebook from your local computer. But if you would like to run it from Google Colab, uncomment and run the lines in the next cell. (*Don't delete the exclamation points at the start of the lines!*) The cell will download and run a shell script that will create subfolders and download the required files for this session.

In [1]:
# !wget -nv https://raw.githubusercontent.com/irs1318dev/pyclass_frc/master/sessions/s20_SQL_IV/get-files-s20.sh
# !bash get-files-s20.sh

### B. Imports and Database File
Run the next cell to set up the notebook to work with our SQLite database.

In [2]:
import sqlite3
import sys

import pandas as pd

# Database file
chinook_db = "chinook.sqlite3"

Since we're going to be connecting to the *Chinook* database over and over, it makes sense create a function that gets a connection, displays the query results, and closes the connection.

In [3]:
def show_query(query, head=None):
    con = sqlite3.connect(chinook_db)
    if head is not None:
        display(pd.read_sql_query(query, con).head(head))
    else:
        display(pd.read_sql_query(query, con))
    con.close()

### C. SQL References
For your convenience, here are the SQL references that were discussed in session 17.
* [Official SQLite documentation](https://www.sqlite.org/lang.html)
* [Python sqlite3 Package Documentation](https://docs.python.org/3/library/sqlite3.html)
* [W3 Schools SQL Tutorial](https://www.w3schools.com/Sql/default.asp) 

## II. Multi-Part Queries
We've seen how we can use aggregate queries to complete sophisticated calculations in SQL. We can complete even more sophisticated calculations in SQL by combining multiple queries.

### A. Nested Queries
SQL allows us to use the output of one query as the input to another query. For example, consider the following query that calculates total sales by country.

In [4]:
# Total Sales by Country
query = """
    SELECT BillingCountry, SUM(Total) AS Sales
      FROM Invoice
  GROUP BY BillingCountry
  ORDER BY Sales DESC;
"""
show_query(query, 10)

,BillingCountry,Sales
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


Suppose we want to calculate the average amount of sales that comes from a single country. In other words, we want to average the contents of the *Sales* column in the preceing query. Unfortunately there is no aggregate function that takes an average of averages. We could get the results we want if we could run an aggregate query on the results from the preceding aggregate query. I'm happy to report that we can do exactly that. It's called a nested query.

In [5]:
# A nested query
query = """
    SELECT AVG(Sales) AS "Average Sales per Country"
    FROM (
            SELECT BillingCountry, SUM(Total) AS Sales
              FROM Invoice
          GROUP BY BillingCountry
          ORDER BY Sales DESC
        );
"""
show_query(query)

,Average Sales per Country
0,97.025


The preceding query is an example of a nested query. Instead of a table name, the `FROM` clause includes an entire `SELECT` query in parentheses. The inner query is the aggregate query that gets the sales total by country. When we nest that query inside the `FROM` clause, SQL treats the output of that query like a table.

The `FROM` clause is not the only place in which we can nest a query. The following query gets sales totals for the three countries that are last in alphabetical order. It does this by nesting a query in the `WHERE` clause.

In [6]:
# Nesting a query in the WHERE clause
query = """
    SELECT BillingCountry, SUM(Total) AS Sales
      FROM Invoice
     WHERE BillingCountry IN (SELECT DISTINCT BillingCountry
                                FROM Invoice
                            ORDER BY BillingCountry DESC
                               LIMIT 3)
  GROUP BY BillingCountry
  ORDER BY Sales DESC;
"""
show_query(query, 10)

,BillingCountry,Sales
0,USA,523.06
1,United Kingdom,112.86
2,Sweden,38.62


### B. `WITH` Clause
Nested queries can be difficult to decipher, especially if there is more than one layer of nesting. Most relational database engines support the `WITH` clause, which makes the queries easier to understand. The following cell contains a re-write of our first nested query. The re-write uses a `WITH` clause.

In [7]:
# A nested query
query = """
    WITH Avg_sales AS (
        SELECT BillingCountry, SUM(Total) AS Sales
          FROM Invoice
         GROUP BY BillingCountry
         ORDER BY Sales DESC
    )
    SELECT AVG(Sales) AS "Average Sales per Country"
    FROM Avg_sales;
"""
show_query(query)

,Average Sales per Country
0,97.025


The `WITH` clause runs a subquery and creates a virtual table from the results. In the preceding example we created the virtual table *Avg_sales*. *Avg_sales* acts just like a regular database table and can be referenced later in the query.

`WITH` queries can have more than one subquery. The following query calculates the fraction of songs in each genre that are love songs. We consider a song to be a love song if it has the word 'love' in the title.

In [8]:
# Genres ranked by fraction of songs that are love songs.
query = """
    WITH Tracks AS (
            SELECT Track.TrackId, Track.Name AS Track, Album.Title AS Album,
                   Artist.Name AS Artist, Genre.Name AS Genre
              FROM Track
        INNER JOIN Album ON Album.AlbumId = Track.AlbumId
        INNER JOIN Genre ON Track.GenreId = Genre.GenreId
        INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
    ),
    Genre_Counts AS (
            SELECT Genre, COUNT() AS song_count
              FROM Tracks
          GROUP BY Genre
    ),
    Love_Counts AS (
            SELECT Genre, COUNT(TrackId) AS love_song_count
             FROM Tracks
            WHERE Track LIKE '%love%'
          GROUP BY Genre
    )
    SELECT Genre_Counts.Genre,
           ROUND(CAST(Love_Counts.love_song_count AS FLOAT) /
                 CAST(Genre_Counts.song_count AS FLOAT), 4) AS "Fraction Love Songs"
      FROM Genre_Counts
 LEFT JOIN Love_Counts ON Genre_Counts.Genre = Love_Counts.Genre
  ORDER BY "Fraction Love Songs" DESC;
"""

show_query(query, 12)

,Genre,Fraction Love Songs
0,Easy Listening,0.1667
1,Pop,0.1042
2,R&B/Soul,0.0984
3,Blues,0.0617
4,Rock,0.0493
5,Reggae,0.0345
6,Electronica/Dance,0.0333
7,Hip Hop/Rap,0.0286
8,Metal,0.0267
9,Alternative,0.0250


Our love-song query has three subqueries. The first subquery, *Tracks*, joins the *Track*, *Artist*, *Genre*, and *Album* tables into a single flat table witout any key values. The *Genre_Counts* subquery counts the songs in each genre, and the *Love_Counts* subquery counts the number of love songs in each genre. Finally, the main query joins the *Genre_Counts* and *Love_Counts* tables to calculate a percentage.

There are a few tricky syntax details for `WITH` queries:
* The `WITH` clause is only placed before the first subquery, no matter how many subqueries there are.
* There is a comma after all but the last subquery.
* Only the final, main query gets a semicolon.

The mentor prefers `WITH` queries to regular nested queries. The queries are easier to read because less indentation is required. More importantly, the name of each subquery describes its purpose. Finally, we can often get a high-level understanding of what the query does by reading only the final, main query. 

### C. Views
One of the problems with `WITH` queries is that they can be difficult to troubleshoot, especially if there is a problem in one of the earlier subqueries. One has to extract the subquery into it's own query to view its output. Sometimes it makes more sense to create a SQL view from one or more of the subqueries.

Consider the first subquery in the love-song query. It displays song tracks and uses joins to replace foreign keys with artist names, genres, and album titles. It's more readable than the *Track* table. Because this query is so useful on its own, it makes sense to create a SQL view from this query.

In [9]:
# A SQL View
# NOTE: You will only be able to run this cell once.
#       If you run it a second time, you will get an error.
query = """
    CREATE VIEW TracksView AS
         SELECT Track.TrackId, Track.Name AS Track, Album.Title AS Album,
                Artist.Name AS Artist, Genre.Name AS Genre
           FROM Track
     INNER JOIN Album ON Album.AlbumId = Track.AlbumId
     INNER JOIN Genre ON Track.GenreId = Genre.GenreId
     INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId;
"""
con = sqlite3.connect(chinook_db)
con.execute(query)
con.commit()
con.close()

The `CREATE VIEW` query didn't generate any visible results. Instead, it created a virtual table called *TracksView* that can be used in later queries as if it were a table.

In [10]:
# Using a VIEW
query = """
    SELECT *
      FROM TracksView
     LIMIT 6;
"""
show_query(query)

,TrackId,Track,Album,Artist,Genre
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,Rock
1,2,Balls to the Wall,Balls to the Wall,Accept,Rock
2,3,Fast As a Shark,Restless and Wild,Accept,Rock
3,4,Restless and Wild,Restless and Wild,Accept,Rock
4,5,Princess of the Dawn,Restless and Wild,Accept,Rock
5,6,Put The Finger On You,For Those About To Rock We Salute You,AC/DC,Rock


No data is copied when we create or use views -- SQL runs the query that the view is based on whenever we reference it in another query. It's like a saved `SELECT` query. Let's create another SQL view for the number of songs in each genre.

In [11]:
# Another SQL View
query = """
    CREATE VIEW IF NOT EXISTS GenreCounts AS
        SELECT Genre, COUNT() AS song_count
          FROM TracksView
         GROUP BY Genre;
"""
con = sqlite3.connect(chinook_db)
con.execute(query)
con.commit()
con.close()

In the preceding query we added an `IF NOT EXISTS` clause. This clause causes the query to do nothing if the view already exists. The clause allows us to re-run the cell without getting an error.

Let's look at the output from our view.

In [12]:
# Querying a view
query = """
    SELECT *
      FROM GenreCounts
     LIMIT 6;
"""
show_query(query)

,Genre,song_count
0,Alternative,40
1,Alternative & Punk,332
2,Blues,81
3,Bossa Nova,15
4,Classical,74
5,Comedy,17


Now that we've created a couple views, our love-song query is much more concise.

In [13]:
# Using views to get genres ranked by fraction of songs that are love songs.
query = """
    WITH Love_Counts AS (
            SELECT Genre, COUNT(TrackId) AS love_song_count
             FROM TracksView
            WHERE Track LIKE '%love%'
          GROUP BY Genre
    )
    SELECT GenreCounts.Genre,
           ROUND(CAST(Love_Counts.love_song_count AS FLOAT) /
                 CAST(GenreCounts.song_count AS FLOAT), 4) AS "Fraction Love Songs"
      FROM GenreCounts
 LEFT JOIN Love_Counts ON GenreCounts.Genre = Love_Counts.Genre
  ORDER BY "Fraction Love Songs" DESC;
"""

show_query(query, 6)

,Genre,Fraction Love Songs
0,Easy Listening,0.1667
1,Pop,0.1042
2,R&B/Soul,0.0984
3,Blues,0.0617
4,Rock,0.0493
5,Reggae,0.0345


SQL views help to reduce repetition in queries. If you find yourself using the same table joins or `WHERE` clauses in multiple queries, consider creating a view. Views are also useful for troubleshooting.

What can be created can also be destroyed. Use a `DROP VIEW` to remove views from a database.

In [15]:
# Removing views
con = sqlite3.connect(chinook_db)
con.execute("DROP VIEW IF EXISTS TracksView;")
con.execute("DROP VIEW IF EXISTS GenreCounts;")
con.commit()
con.close()

Unlike other relational databases, SQLite does not allow modifying views that already exist. If you must modify a view, drop it and re-create it.

Check out the SQLite documentation for additional information on [CREATE VIEW queries](https://www.sqlite.org/lang_createview.html) and [DROP VIEW queries](https://www.sqlite.org/lang_dropview.html)

### D. Exercises 1 - 8

#### Ex. #1
Write a nested query that calculates the average length, minimum length, and maximum length of the playlists that are included in the database.

**HINT:** Start with an aggregate query that counts the number of tracks in each playlist.

In [ ]:
# Ex. #1
query = """
    -- Write query here.   
          
"""
show_query(query)

#### Ex. #2
The average playlist length from exercise #8 should be 622.5 tracks. Carefully consider your join type if you did not get that answer.
* What type of join should be used in this query?
* Why does changing the join type change the results for average playlist length? Be specific.

In [ ]:
# Ex. #2
#
#

#### Ex. #3
Convert the query in exercise #8 to a `WITH` query.

In [ ]:
# Ex. #3
#
#

#### Ex. #4
Write a `WITH` query that shows the playlist name, album title, and track name for the five playlists with the fewest number of songs.

In [ ]:
# Ex. #4
query = """
    -- Write query here.   

"""

show_query(query)

#### Ex. #5
Create a view that shows the Playlist name, album name, track name, and artist name for every track that occurs in a playlist.
* The view's column names shold be Playlist, Album, Track, and Artist
* Order the results by playlist name then track name.
* Display the top 12 rows from the view.

In [ ]:
# Ex. #5




#### Ex. #6
Views are included in the sqlite_master (or *sqlite_schema*) table. Show the *PlaylistTrackView* in the *sqlite_master* table.
* **HINT:** Set the *type* column equal to 'view'.

In [ ]:
# Ex #6
query = """
        -- Write query here.
        
"""
show_query(query)

#### Ex. #7
Drop the view created in exercise #12.

In [ ]:
# Ex #7



#### Ex. #8
Write a query that finds the album with the higest percentage of love songs.

In [ ]:
# Ex #8
#
#

## III. Window Queries

**NOTE:** None of the window queries will work in Google Colab. Window queries were added to SQLite in version 3.25 and Google Colab used version 3.22 at the time this notebook was written.

Before we discuss window queries, let's review aggregate queries, which were covered in session 19. The following aggregate query calculates the average track length in seconds by genre.

In [16]:
# An aggregate query
query = """
    SELECT Genre.Name AS Genre, AVG(Track.Milliseconds / 1000) AS Seconds
      FROM Track
INNER JOIN Genre
        ON Genre.GenreId = Track.GenreId
  GROUP BY Genre;
"""
# The following line tells Pandas how we want our floating point values
#   to be formatted.
with pd.option_context('display.float_format', lambda x: "{:,.1f}".format(x)):
    show_query(query, 6)

,Genre,Seconds
0,Alternative,263.6
1,Alternative & Punk,233.9
2,Blues,269.8
3,Bossa Nova,219.0
4,Classical,293.4
5,Comedy,"1,584.8"


The raw *Track* table contains over 3,500 rows. If we were to display the entire results from the preceding query, we would see 24 rows, one row for each genre that appears in the *Genre* table. The data for each row in our output is calculated from mutiple rows using a summary statistic, such as `AVG()`. The query *aggregates* the data from multiple rows into a single row.

None of this should be new -- we're just reviewing this principle so we can contrast aggregate queries with window queries. Now let's look at a window query. Suppose we wanted to rank the songs within their genres by length.

In [17]:
# A window query
# WARNING: will not run in Google Colab

query = """
    SELECT Genre.Name As Genre, Track.Name AS Track,
           Milliseconds / 1000 AS Seconds,
           RANK() OVER (PARTITION BY Genre.Name ORDER BY Milliseconds DESC) AS "Genre Rank"
      FROM Track
INNER JOIN Genre
        ON Track.GenreId = Genre.GenreId
        ORDER BY Seconds DESC;
"""
# The following line tells Pandas how we want our floating point values
#   to be formatted.
with pd.option_context('display.float_format', lambda x: "{:,.1f}".format(x)):
    show_query(query)

,Genre,Track,Seconds,Genre Rank
0,TV Shows,Occupation / Precipice,5286,1
1,Drama,Through a Looking Glass,5088,1
2,Sci Fi & Fantasy,"Greetings from Earth, Pt. 1",2960,1
3,Sci Fi & Fantasy,The Man With Nine Lives,2956,2
4,Sci Fi & Fantasy,"Battlestar Galactica, Pt. 2",2956,3
...,...,...,...,...
3498,Hip Hop/Rap,Commercial 1,7,35
3499,Alternative & Punk,Oprah,6,330
3500,Alternative & Punk,A Statistic,6,331
3501,Alternative & Punk,Now Sports,4,332


The preceding query contains 3,503 rows, one for each record in the *Track* table. The first three columns are typical columns one might find in a `SELECT` query. But the final column, *Genre Rank*, is different. If a track is the longest track within it's genre, then *Genre Rank* is 1. If it's the second longest track within its genre, then *Genre Rank* is 2, and so on. If we wanted, we could include this query in a composite query and find the 5th longest track in each genre.

In [ ]:
# Find the 5th longest track for each genre
# WARNING: will not run in Google Colab
query = """
    WITH TrackRanks AS (
        SELECT Genre.Name As Genre, Track.Name AS Track,
               Milliseconds / 1000 AS Seconds,
               RANK() OVER (PARTITION BY Genre.Name ORDER BY Milliseconds DESC) AS "Genre Rank"
          FROM Track
    INNER JOIN Genre
            ON Track.GenreId = Genre.GenreId
            ORDER BY Seconds DESC)
    
    SELECT *
      FROM TrackRanks
     WHERE "Genre Rank" = 5;
"""
# The following line tells Pandas how we want our floating point values
#   to be formatted.
with pd.option_context('display.float_format', lambda x: "{:,.1f}".format(x)):
    show_query(query, 8)

The *Genre Rank* column is defined by the following clause:
```SQL
RANK() OVER (PARTITION BY Genre.Name ORDER BY Milliseconds DESC) AS "Genre Rank"
```
* `RANK()` is a window function.
* The `OVER ()` clause is what makes this query a window query. It defines the subset of rows, or *window*, over which the `RANK()` function operates. The preceding `OVER()` clause has two parts.
  * The `PARTITION BY Genre.Name` operates similar to how `GROUP BY Genre.Name` would operate in an aggregate query. It causes `RANK()` to calculate different rankings within each genre.
  * The `ORDER BY Milliseconds DESC` clause causes the rows to be ordered and ranked by the contents of the *Milliseconds* column. 
  
The `ORDER BY` clause is mandatory when using the `RANK()` function because the `RANK()` function doesn't make any sense if there is no defined order. But we could remove the `PARTITION BY` clause if we wanted.

In [ ]:
# Window query with no PARTITION BY clause
# WARNING: will not run in Google Colab
query = """
    SELECT Genre.Name As Genre, Track.Name AS Track,
           Milliseconds / 1000 AS Seconds,
           RANK() OVER (ORDER BY Milliseconds DESC) AS "Genre Rank"
      FROM Track
INNER JOIN Genre
        ON Track.GenreId = Genre.GenreId
        ORDER BY Seconds DESC;
"""
# The following line tells Pandas how we want our floating point values
#   to be formatted.
with pd.option_context('display.float_format', lambda x: "{:,.1f}".format(x)):
    show_query(query)

Without the `PARTITION BY` clause, a single ranking is generated for the entire table, ranging from 1 for the longest track to 3503 for the shortest.

Figure 1 graphically depicts the difference between aggregate and window queries.

#### Figure 1
![Window Query](images/window-query.png)

Aggregate functions can also be used in window queries. The next query calculates the cumulative sum by date for sales during the first three months of 2009.

In [ ]:
# Calculating cumulative sums
# WARNING: will not run in Google Colab
query = """
    SELECT DATE(InvoiceDate) AS InvoiceDate,
           STRFTIME('%Y-%m', InvoiceDate) AS Month,
           Total,
           SUM(Total) OVER(
               PARTITION BY STRFTIME('%Y-%m', InvoiceDate)
                   ORDER BY InvoiceDate 
               ROWS BETWEEN UNBOUNDED PRECEDING
                        AND CURRENT ROW)
                         AS CumulativeSum
      FROM Invoice
     WHERE Month IN ('2009-01', '2009-02', '2009-03')
  ORDER BY InvoiceDate;
"""
show_query(query)

The preceding query introduces two new concepts: date functions and frame specifications.

### A. Date Functions
The `DATE()` and `STRFTIME()` function are SQLite date functions. Unlike most other relational databases, SQLite does not have a date datatype, but it does have a few date functions. Dates and times can be stored as text, integers, or real numbers. If stored as text, the dates and times must adhere to the format `YYYY-MM-DD HH:MM:SS:SSS`, which corresponds to years, months, days, hours, minutes, seconds, and fractional seconds, in that order. Also, the hours must be in 24-hour time.

The `DATE` function extracts the date portion of the *InvoiceDate* column and the `STRFTIME()` function uses a format string to extract the year and month from the *InvoiceDate* column. See [the SQLite documentation on date and time functions](https://www.sqlite.org/lang_datefunc.html) for more information.

The SQLite `JULIANDAY()` function is worth noting. It converts dates to an integer. The integer is the number of days since noon in Greenwich on November 24th, 4714 BCE (before common era). The 4174 BCE start date is based on work by Joseph Scaliger, a classical scholar who lived from 1540 to 1609. 

In [ ]:
# JULIANDAY Function
query = """
    SELECT CAST(JULIANDAY(InvoiceDate) AS Integer) AS InvoiceDate, Total
      FROM Invoice
  ORDER BY InvoiceDate;
"""
show_query(query, 6)

The `JULIANDAY()` function is surprisingly useful. Its output is meaningless to most humans, but it makes it easy to calculate the number of days between two dates, or to join tables by date.

### B.Frame Specification with `ROWS`
`OVER()` clauses, which are also called window definitions, can have up to three parts: the `PARTITION BY` clause, the `ORDER BY` clause, and the frame specification. The frame specification defines a set or rows, or window frame, that the window or aggregate function operates on.

In the earlier window query, we used the `SUM()` function to calculate the sum of the values in the *Total* column, but we didn't want to sum all rows in the partition. We only want to sum rows that occur between the first row of the partition and the current row. We accomplished this by adding a frame specification to our window definition.

The clause `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW` instructs SQL to sum only the rows that occur between the very first row (i.e., `UNBOUNDED PRECEDING`) and the current row.

It's unlikely anyone would want to do this, but we could also calculate a shrinking sum that starts with the total sales for the month and descends to zero. Note that we are using the phrase `UNBOUNDED FOLLOWING` to refer to the final row of the partition.

In [ ]:
# Limiting the window to specific rows
# WARNING: will not run in Google Colab
query = """
    SELECT DATE(InvoiceDate) AS InvoiceDate,
           STRFTIME('%Y-%m', InvoiceDate) AS Month,
           Total,
           SUM(Total) OVER(
               PARTITION BY STRFTIME('%Y-%m', InvoiceDate)
                   ORDER BY InvoiceDate 
               ROWS BETWEEN CURRENT ROW
                        AND UNBOUNDED FOLLOWING)
                         AS DescendingSum
      FROM Invoice
     WHERE Month IN ('2009-01', '2009-02')
  ORDER BY InvoiceDate;
"""
show_query(query)

We could get really wierd and calculate an average total from the frame consisting of the current row and the three rows that occur before and after the current row (seven
rows total).

In [ ]:
# Calculating 7-day average with ROWS
# WARNING: will not run in Google Colab
query = """
    SELECT DATE(InvoiceDate) AS InvoiceDate,
           STRFTIME('%Y-%m', InvoiceDate) AS Month,
           Total,
           AVG(Total) OVER(
               PARTITION BY STRFTIME('%Y-%m', InvoiceDate)
                   ORDER BY InvoiceDate 
               ROWS BETWEEN 3 PRECEDING
                        AND 3 FOLLOWING)
                         AS "7-Day Average"
      FROM Invoice
     WHERE Month IN ('2009-01', '2009-02')
  ORDER BY InvoiceDate;
"""
show_query(query)

### C.Frame Specification with `RANGE`
Replacing `ROWS` with the `RANGE` keyword causes SQL to include rows whose *values* fall within the specified limits. The next query counts the number of invoices within the same month that have a total within $1 of the current row.

In [ ]:
# Invoices within $1 of the current row.
# WARNING: will not run in Google Colab
query = """
    SELECT DATE(InvoiceDate) AS InvoiceDate,
           STRFTIME('%Y-%m', InvoiceDate) AS Month,
           Total,
           COUNT(*) OVER(
               PARTITION BY STRFTIME('%Y-%m', InvoiceDate)
                   ORDER BY Total
               RANGE BETWEEN 1 PRECEDING
                        AND 1 FOLLOWING)
                         AS "Count of totals within $1"
      FROM Invoice
     WHERE Month IN ('2009-01', '2009-02')
  ORDER BY InvoiceDate;
"""
show_query(query)

In addition to replacing `ROWS` with `RANGE`, we had to order the window by the *Total* column.

### D. `LEAD()` and `LAG()`
The `LEAD()` and `LAG()` functions will retrieve values from nearby rows.

In [ ]:
# LAG() example
# WARNING: will not run in Google Colab
query = """
    SELECT DATE(InvoiceDate) AS InvoiceDate,
           BillingCity,
           LAG(Total) OVER(ORDER BY InvoiceDate) AS "Prior Total",
           Total AS "Current Total"
      FROM Invoice;        
"""
show_query(query, 8)

Both `LEAD()` and `LAG()` accept a row offset value. The next query shows the total from the row that is three rows ahead.

In [ ]:
# LEAD() example
# WARNING: will not run in Google Colab
query = """
    SELECT DATE(InvoiceDate) AS InvoiceDate,
           BillingCity,
           Total AS "Current Total",
           LEAD(Total, 3) OVER(ORDER BY InvoiceDate) AS "Future Total"
      FROM Invoice;        
"""
show_query(query, 8)

### E. Ranking Functions
We already used one of the ranking functions, `RANK()` in a query. Run the next query to see how the `RANK()` functions behaves when there is a tie. The query ranks employees by the year they were hired.

In [ ]:
# RANK() Example
# WARNING: will not run in Google Colab
query = """
    SELECT FirstName || ' ' || LastName AS Name,
           STRFTIME('%Y', HireDate) AS HireYear,
           RANK() OVER (ORDER BY STRFTIME('%Y', HireDate)) AS "Seniority Rank"
      FROM Employee;
"""
show_query(query)

There were three employees that were hired in 2002 and tied for the highest seniority. They were all assigned a *Seniority Rank* of 1. Since there was a three-way tie for first, the employees who were hired in 2003 were all assigned a *Seniority Rank* of 4.

There is similar function called `DENSE_RANK()` that handles ties differently.

In [ ]:
# DENSE_RANK() Example
# WARNING: will not run in Google Colab
query = """
    SELECT FirstName || ' ' || LastName AS Name,
           STRFTIME('%Y', HireDate) AS HireYear,
           DENSE_RANK() OVER (ORDER BY STRFTIME('%Y', HireDate)) AS "Seniority Rank"
      FROM Employee;
"""
show_query(query)

`DENSE_RANK()` assigns the same rank to rows with the same value, but it does not skip any rank values. Even though there is a three-way tie for first, all employees hired in 2003 are still assigned a rank of 2.

Sometimes we don't care about ties. We just want to assign an increasing integer to each row. `ROW_NUMBER()` is the right function for this situation.

In [ ]:
# ROW_NUMBER() Example
# WARNING: will not run in Google Colab
query = """
    SELECT FirstName || ' ' || LastName AS Name,
           STRFTIME('%Y', HireDate) AS HireYear,
           ROW_NUMBER() OVER (ORDER BY STRFTIME('%Y', HireDate)) AS "Seniority Rank"
      FROM Employee;
"""
show_query(query)

### F. Named Window Definitions
What if you wanted to have two columns in your query that used the same window definition? You could just repoeat the window specification for each column, but that get's wordy.

In [ ]:
# Repeated Window Specification
# WARNING: will not run in Google Colab
query = """
    SELECT DATE(InvoiceDate) AS InvoiceDate,
           BillingCity,
           LAG(Total) OVER (ORDER BY InvoiceDate) AS "Prior Total",
           Total AS "Current Total",
           LEAD(Total) OVER (ORDER BY InvoiceDate) AS "Next Total"
      FROM Invoice;        
"""
show_query(query, 6)

SQLite allows users to define and name a window specification with a `WINDOW` clause. The named window specification can be used multiple times within the same query.

In [ ]:
# Named Window Specification
# WARNING: will not run in Google Colab
query = """
    SELECT DATE(InvoiceDate) AS InvoiceDate,
           BillingCity,
           LAG(Total) OVER InvoiceWin AS "Prior Total",
           Total AS "Current Total",
           LEAD(Total) OVER InvoiceWin AS "Next Total"
      FROM Invoice
    WINDOW InvoiceWin AS (ORDER BY InvoiceDate);        
"""
show_query(query, 6)

### G. Window Query References
Scan the [SQLite's official documentation on window queries](https://www.sqlite.org/windowfunctions.html). It precisely explains the syntax that is accepted by SQLite and lists all window functions and related key words. SQLite's documentation describes many features of window queries that were not explained in this tutorial.

If you are having trouble understanding window queries, try reading through the [window query tutorial on www.sqltutorial.org.](https://www.sqltutorial.org/sql-window-functions/)

### H. Exercises 9 - 11

#### Ex. #9
Draft a query that returns the track name, genre, bytes, and the average number of bytes for all tracks within a specific genre. Order the results by bytes, in ascending order.

In [ ]:
# Ex. #9



#### Ex. #10
Write a query that calculates a cumulative sum of bytes for every album. The query should include columns for the track, album, Mebibytes, and cumulative sum of Mebibytes. Order the tracks by *TrackId* within the window definition. Order the results by album title and *TrackId*.

**HINTS:**
* Divide bytes by 1048576 to convert to mebibytes.
* The *Bytes* column has a datatype of integer. More precise results can be obtained by casting *Bytes* to a FLOAT value, i.e., `CAST(Track.Bytes AS FLOAT)`.

In [ ]:
# Ex. #10



#### Ex. #11
Write a query that shows the invoice date and total dollar amount for each invoice. Include a column that contains the 7-day average total. That is, the column should contain the average of the *Total* column for the period starting six days prior to the current date. Order the results by *InvoiceDate*.

**HINTs**
* Don't forget about the `JULIANDAY()` function. Use it in the `ORDER BY` section of the `OVER` clause.
* Use the `DATE()` function to remove time components from *InvoiceDate*.

In [ ]:
# Ex. #11




## V. Exercises 12 - 13

#### Ex. #12
Write a query that lists the artists with the 2nd-higest number of tracks for each genre.
* The query should return four columns: genre, artist, tracks, and the artists rank by number of tracks, i.e., the artist with the most tracks in a genre will have rank 1.
* In case of ties, show all artists who tied for first or second.
* Order the results by genre then rank.
* **HINT:** The mentor's solution required a `WITH` query with two subqueries and a main query.

In [ ]:
# Ex. #12



#### Ex. #13
Write a query that calculates the population variance of the track lengths, in seconds, for each genre.
* The results should have two columns, genre and variance.
* The results should be sorted by variance in descending order.
* Display the variances as integers.

**HINTS**
* The formula for variance is:
$$\textrm{Var} = \sum^n_i \frac{(x_i - \bar{x})^2}{n}$$
  * $x_i$ is the length of track $i$.
  * $\bar{x}$ is the average track length for all tracks in a genre.
  * $n$ is the number of tracks in the genre.
  * In English, we want to calculate the average track length for each genre. Then for every track, we'll need to square the difference between the track's length and the average track length, sum the squared differences for each track, and finally divide the sum by the number of tracks in each genre.
* This will require a composite query.
  * Start by joining the *Track* and *Genre* tables and converting millisecons to seconds.
  * Add a column *to each row* with the average track length for the applicable genre (requires window query).
  * Calculate the square of the difference between the track length and average track length.
  * Sum the squared differences and divide by the number of tracks in each genre (requires aggregate query).
* With respect to displaying results as an integer, don't forget about the `CAST()` function

In [ ]:
# Ex. #13



## VI. Optional HackerRank Query
SQL queries can get long and complicated. If you would like to see an example of how a complicated query can be built up in small steps, check out the [15 Days of SQL Query from HackerRank](SQL15Days/SQL15Days.ipynb).

## VII. More to Learn
We've covered many features and capabilities of Structured Query Language (SQL) in this notebook and the preceding three notebooks. But make no mistake, there is much more to learn about SQL. This class focused on commonly used SQL features and features that we expect to use in our robotics scouting system. Here is a short and incomplete list of features we *did not* cover.
* [These notebooks covered only a fraction of SQLite's functions.](https://www.sqlite.org/lang_corefunc.html)
* For every type of query we covered, there are options and features that we didn't have time to discuss.
* [Python's *sqlite3* package has features that we did not discuss.](https://docs.python.org/3/library/sqlite3.html)
* Triggers are database operations that are automatically performed when certain events occur. Triggers can automatically run update, insert, or delete queries when tables are altered. [See SQLite's documentation on triggers for more information.](https://www.sqlite.org/lang_createtrigger.html)
* [SQLite INSERT, UPDATE, and DELETE queries can be modified to return the addeded, updated, or deleted data.](https://www.sqlite.org/lang_returning.html)
* Other database systems have even more features. Some systems allow users to temporarily store values in variables. Others have queries that create custom functions.
* There are many optimization techniques that will make queries on large datasets run faster.

In summary, it's a good idea to explore the documentation for whatever database system you are using. If you are having a difficult time implementing a feature in your program, or your code seems more complicated than it should be, there is a good chance that there is a SQL feature that would make your work easier.

## VIII. Save Your Work
Once you have completed the exercises, save a copy of the notebook outside of the git repository (outside of the *pyclass_frc* folder). Include your name in the file name. Send the notebook file to another student to check your answers.

## IX. Concept and Terminology Review
You should be able to define the following terms or describe the concept. 
* Nested Query
* `WITH`
* Views
* `CREATE VIEW`
* Window Query
* Window Definition
* `OVER`
* `PARTITION BY`
* `ORDER BY`
* Frame Specification
* `RANK()`
* `DENSE_RANK()`
* `ROW_NUMBER()`
* `LAG()`
* `LEAD()`
* `ROWS`
* `RANGE`
* `UNBOUNDED PRECEDING`
* `UNBOUNDED FOLLOWING`
* `CURRENT ROW`
* `DATE()`
* `STRFTIME()`
* `JULIANDAY()`
* `WINDOW`


[Table of Contents](../../index.ipynb)